<a href="https://colab.research.google.com/github/rekkerz/X-AI-workshops/blob/master/X_AI_Workshop2_Python_Gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependancies, takes around 45 seconds

Rendering Dependancies



In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Installing Dependancies

In [0]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

# Imports and Helper functions


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from gym import wrappers

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Cartpole!

In [0]:
env = gym.make("CartPole-v1") # Here we would select the environment
env = wrap_env(env)           # We need to wrap the environment to use it in Google Colab.

In [0]:
print(env.action_space)       # This shows us the action space. In case of cartpole you can move left or right so action space is 2.
print(env.observation_space)  # This shows the observation space.

Let's see how the environment works by using it with a default movement.

In [0]:
episodes = 100
observation = env.reset()

for _ in range(episodes):
    env.render()
    action = env.action_space.sample() # This is a random movement at the moment.
    observation, reward, done, info = env.step(action)
    
    if done:
      print("Observation = ",observation)
      print("Info = ",info)
      break;

env.close()
show_video()

# Approaches

Currently the pole makes a random movement out it's action space. The goal of this simulation is to balance the pole. There are many approaches to this (some of them won't be covered).

## Moving left and right repeatedly

In this approach we will simply move left and right repeatedly.

In [0]:
env = wrap_env(gym.make("CartPole-v1"))

observation = env.reset()
right = True
for _ in range(1000):
  env.render()

  if(right == True):
    action = 1
    right = False
  else:
    action = 0
    right = True
    
  observation, reward, done, info = env.step(action)

  if done:
    observation = env.reset()

env.close()
show_video()

### Using Weight Vector

The idea of this approach revolves around the position of the pole.

Action = 1 if np.dot(observation,new_weights) > 0 else 0

In [0]:
bestLength = 0
episode_length = []
best_weights = np.zeros(4)    # Generates an array of 4 zeros.
flag = 0
max_life = 1000

In [0]:
env = wrap_env(gym.make("CartPole-v1"))

for i in range(1):   # There will be 1 trial.
  new_weights = np.random.uniform(-1,1,4) # generates an array of 4 int between in range(-1,1)
  length = []
  for j in range(500):  # There will be 500 timesteps 
    observation = env.reset()
    done = False
    count = 0
    while not done:
      count += 1
      action = 0 if np.dot(observation,new_weights) >0 else 1  
      # This defines the method used. Will be explained by presenter.
      observation,reward,done,_ = env.step(action)
      if done:
        break
      elif count > max_life:
        flag = 1
        break
    length.append(count)
  avg_length = float(sum(length)/ len(length))
  print("Average length = ", avg_length)

env.close()
show_video()

In [0]:
env= wrappers.Monitor(env, 'brute_force_files', force = True)
done=  False
count = 0
observation = env.reset()

while not done:
    count = count +1
    action = 1 if np.dot(observation,best_weights) >0 else 0
    observation,reward,done,_ = env.step(action)
    if done:
        break
print('with best weights, game lasted ',count , ' moves')

# Neural Network


A neural network will be deeper cover in the next lecture.